#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 5, 11)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,16,S,RJH,"4,500",33.25,2,CP1S,SET,2022-02-02


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues')

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
1,S,DOHOME,2,CP1S,SET100,2400,19.800000,19.600000,0.300000,+1.55%,2,2022-02-02,Sell,47520.000000
7,S,ORI,10,CP1S,SET100,15000,11.100000,10.100000,0.450000,+4.66%,2,2022-05-09,Sell,166500.000000
8,S,PTT,2,C38.50,SET50,3000,37.500000,37.000000,0.000000,0.00%,2,2022-03-03,Sell,112500.000000
9,S,RATCH,4,CUT,SET50,2000,41.000000,40.000000,-0.750000,-1.84%,2,2022-03-17,Sell,82000.000000
11,S,RJH,3,CP1S,SET,4500,33.250000,32.500000,0.750000,+2.36%,2,2022-02-02,Sell,149625.000000


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index()

trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
S,BCH,18,CP1S,SET100,3000,21.400000,19.600000,-1.200000,-5.77%,0,2022-05-10,Sell,64200.000000
S,KCE,17,C62.75,SET50,1000,64.000000,59.750000,0.500000,+0.84%,0,2022-03-21,Sell,64000.000000
S,HREIT,12,DOS,SET,40100,9.000000,8.400000,0.000000,0.00%,0,2022-03-04,Sell,360900.000000
S,IP,12,CP1S,mai,2500,20.300000,19.100000,0.300000,+1.60%,0,2022-02-02,Sell,50750.000000
S,DOHOME,2,CP1S,SET100,2400,19.800000,19.600000,0.300000,+1.55%,2,2022-02-02,Sell,47520.000000


### End of Tables in the process

### Print to verify before upload file

In [17]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [18]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [19]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,SENA,"15,000",3.88,"58,200.00",RD15pct,SET100,2,2022-02-02
1,S,DOHOME,"2,400",19.80,"47,520.00",CP1S,SET100,2,2022-02-02
2,S,PTT,"3,000",37.00,"111,000.00",C38.50,SET50,2,2022-03-03
3,S,RATCH,"2,000",41.00,"82,000.00",CUT,SET50,2,2022-03-17
4,S,RJH,"4,500",33.25,"149,625.00",CP1S,SET,2,2022-02-02


In [20]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B     58200.0
S    390145.0
Name: amount, dtype: float64

In [21]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
0,B,SENA,"15,000",3.88,"58,200.00",RD15pct,SET100,2,2022-02-02


In [22]:
cash = 194_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(194000, 58200.0, 135800.0)

In [23]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active,xdate
1,S,DOHOME,"2,400",19.80,"47,520.00",CP1S,SET100,2,2022-02-02
2,S,PTT,"3,000",37.00,"111,000.00",C38.50,SET50,2,2022-03-03
3,S,RATCH,"2,000",41.00,"82,000.00",CUT,SET50,2,2022-03-17
4,S,RJH,"4,500",33.25,"149,625.00",CP1S,SET,2,2022-02-02


In [24]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(194000, 390145.0, 584145.0)

### After call ord-log (must call everytime that orders change)

In [25]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [26]:
show(df[['trans','name','spd','reason','qty','target','current','active']])

trans,name,spd,reason,qty,target,current,active


In [27]:
mb = (df.trans == 'Buy') & (df.spd >= -8)
df[mb].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,SENA,-8,RD15pct,SET100,15000,3.88,4.04,-0.28,-6.48%,2,2022-02-02,Buy,58200.0


In [28]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [29]:
ms = (df.trans == 'Sell') & (df.spd <= 7)
df[ms].sort_values(['name'],ascending=[True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
2,S,DOHOME,7,CP1S,SET100,2400,19.80,19.10,-0.50,-2.55%,2,2022-02-02,Sell,47520.0
9,S,PTT,1,C38.50,SET50,3000,37.00,36.75,-0.25,-0.68%,2,2022-03-03,Sell,111000.0
10,S,RATCH,6,CUT,SET50,2000,41.00,39.50,-0.50,-1.25%,2,2022-03-17,Sell,82000.0
12,S,RJH,6,CP1S,SET,4500,33.25,31.75,-0.75,-2.31%,2,2022-02-02,Sell,149625.0


In [30]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate,trans,amount
0,B,SENA,-8,RD15pct,SET100,15000,3.88,4.04,-0.28,-6.48%,2,2022-02-02,Buy,58200.0
2,S,DOHOME,7,CP1S,SET100,2400,19.80,19.10,-0.50,-2.55%,2,2022-02-02,Sell,47520.0
9,S,PTT,1,C38.50,SET50,3000,37.00,36.75,-0.25,-0.68%,2,2022-03-03,Sell,111000.0
10,S,RATCH,6,CUT,SET50,2000,41.00,39.50,-0.50,-1.25%,2,2022-03-17,Sell,82000.0
12,S,RJH,6,CP1S,SET,4500,33.25,31.75,-0.75,-2.31%,2,2022-02-02,Sell,149625.0


In [31]:
df[mb | ms].shape[0]

5

In [32]:
mask = (df.trade == 'B')
df[mask].nlargest(5, "amount")[["name","trans","amount","spd"]].style.format(format_dict)

,name,trans,amount,spd
0,SENA,Buy,"58,200.00",-8


### Select source of orders between these two

In [33]:
df_out = df[mb | ms][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,SENA,15000,3.88,2,-8,4.04,-0.28,-6.48%,RD15pct,SET100,2022-02-02
2,Sell,DOHOME,2400,19.80,2,7,19.10,-0.50,-2.55%,CP1S,SET100,2022-02-02
9,Sell,PTT,3000,37.00,2,1,36.75,-0.25,-0.68%,C38.50,SET50,2022-03-03
10,Sell,RATCH,2000,41.00,2,6,39.50,-0.50,-1.25%,CUT,SET50,2022-03-17
12,Sell,RJH,4500,33.25,2,6,31.75,-0.75,-2.31%,CP1S,SET,2022-02-02


In [34]:
df_out = df[df.active == 2][colt]
df_out

,trans,name,qty,target,active,spd,current,change,percent,reason,market,xdate
0,Buy,SENA,15000,3.88,2,-8,4.04,-0.28,-6.48%,RD15pct,SET100,2022-02-02
2,Sell,DOHOME,2400,19.80,2,7,19.10,-0.50,-2.55%,CP1S,SET100,2022-02-02
9,Sell,PTT,3000,37.00,2,1,36.75,-0.25,-0.68%,C38.50,SET50,2022-03-03
10,Sell,RATCH,2000,41.00,2,6,39.50,-0.50,-1.25%,CUT,SET50,2022-03-17
12,Sell,RJH,4500,33.25,2,6,31.75,-0.75,-2.31%,CP1S,SET,2022-02-02


In [35]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process